In [60]:
# O objetivo final é identificar os algarismos na foto e retornar uma imagem quadrada
# de tamanho padrão com cada um deles centralizados.
#
# Atenção: quando a imagem aparecer não feche a janela, aperte alguma tecla ou então
# o jupyter da erro.
#
# Falta:
# - atualmente o código retorna um retângulo que 'encosta' no algarismo, falta
#   formar uma imagem quadrada, centralizada e padronizar o tamanho.

In [61]:
import numpy as np
import cv2

In [62]:
# deixa a imagem em escala de cinza e borra para reduzir o número de contornos
img = cv2.imread('teste2.jpeg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray,(5,5),0)

In [63]:
# binariza e plota
thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 135, 5)
cv2.imshow('binarizada', thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [64]:
# identifica contornos na imagem
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

In [65]:
# calcula as áreas dos retângulos construídos pelos contornos em contours
# o retângulo é limitado entre x e x+h na horizontal e entre y e y+w na vertical
area = np.zeros(len(contours))
for i, ctr in enumerate(contours):
    x, y, w, h = cv2.boundingRect(ctr)
    area[i] = w*h
bigarea = np.amax(area[0:-1])
# bigarea é a area do maior dígito, area[-1] corresponde a toda a imagem e por isso é desprezado

In [66]:
# imaginando que os algarismos são os maiores contornos, é suposto que
# todo contorno de tamanho superior a 15% do maior corresponde a um algarismo
for i, ctr in enumerate(contours):
    x, y, w, h = cv2.boundingRect(ctr)
    if 0.15*bigarea < area[i]:
        cv2.imwrite("test_results/"+str(i)+".jpg",thresh[y:y+h,x:x+w])
        cv2.rectangle(thresh, (x, y), (x + w, y + h), (0, 255, 0), 2)

In [67]:
# plota a imagem mostrando os cortes
cv2.imshow('cortes', thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()